<a href="https://colab.research.google.com/github/ahairshi/dsf-redis/blob/main/locust_task_name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Got it. You want to change the **Locust task name** used for reporting.

In Locust, the task name that appears in the Web UI and reports is determined by the `name` argument passed to the HTTP method (`self.client.post`, `self.client.get`, etc.). If you omit the `name` argument, it defaults to the request path.

Here is the modified script, changing the task name to a specific value: **`Submit_Order_XML_Request`**.

### Locust Script with Custom Task Name (`locustfile.py`)

In [ ]:
import os
import random
from locust import FastHttpUser, task, between, events
from typing import Dict, List, Any

# ==============================================================================
# 1. Configuration & Data Loading
# ==============================================================================

# Define the list of XML payload files to load
XML_FILES: List[str] = [
    "payload_a.xml",
    "payload_b.xml"
]

# This will store the content of the XML files
PAYLOADS: List[str] = []

# Define the headers (replace with your actual cURL headers)
HEADERS: Dict[str, str] = {
    'Content-Type': 'text/xml; charset=utf-8',
    'SOAPAction': '"http://tempuri.org/MethodName"',
    'Accept-Encoding': 'gzip,deflate'
}

# --- Event Hook to Load Files Before Test Starts ---
@events.init.add_listener
def _(environment, **kwargs):
    """Load all XML payload files into memory when Locust starts."""
    for file_name in XML_FILES:
        try:
            file_path = os.path.join(os.path.dirname(__file__), file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                PAYLOADS.append(f.read())
            print(f"INFO: Successfully loaded payload from {file_name}")

        except FileNotFoundError:
            print(f"ERROR: Payload file not found: {file_name}")
        except Exception as e:
            print(f"ERROR: Could not read file {file_name}: {e}")

# ==============================================================================
# 2. Locust User Class Definition using FastHttpUser
# ==============================================================================

class XmlServiceUser(FastHttpUser):
    """Simulates a user making requests with a randomly selected XML payload."""

    host = "http://localhost:8080"
    wait_time = between(1, 3)

    @task
    def submit_random_xml_request(self):
        """
        Executes a POST request with a randomly chosen XML payload.
        """

        if not PAYLOADS:
            print("ERROR: No payloads were loaded. Skipping task.")
            return

        selected_payload = random.choice(PAYLOADS)
        endpoint_path = "/api/xml-processor"

        # -------------------------------------------------------------------
        # THE LOCUST TASK NAME IS SET HERE USING THE 'name' PARAMETER
        # -------------------------------------------------------------------
        response = self.client.post(
            endpoint_path,
            data=selected_payload,
            headers=HEADERS,
            # This static string will be the name reported in the statistics table
            name="Submit_Order_XML_Request"
        )

        # --- Optional: Response Validation ---
        if response.status_code != 200:
            response.failure(f"Request failed with status code: {response.status_code}")

### Explanation of the Change

The key change is in the **`self.client.post`** call:

In [ ]:
name="Submit_Order_XML_Request"

By providing a static string to the `name` argument, all requests made by this task will be grouped under **`Submit_Order_XML_Request`** in the Locust statistics, regardless of the `endpoint_path` or the specific XML payload used. This is the standard way to control task reporting names in Locust.